In [1]:
# would recommend using conda for the packages
import sys
import os
import io
from pgn_parser import parser, pgn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, History
import chess
import chess.pgn
import tqdm
import pandas as pd
import numpy as np
import random
from stockfish import Stockfish


In [2]:
def board_fen_to_8x8(bf):
    our_board = list()
    long_board = list()
    long_board2 = list()
    for s in bf.split('/'):
        long_board+=s
    for l in long_board: 
        if l.isnumeric():     
            i = int(l)
            while i>0:
                long_board2.append(0)
                i-=1
        else:
            long_board2.append(l)
    h =0
    for i in range(0,8):
        our_board.append([])
        for j in range(0,8):
            our_board[i].append([])
            our_board[i][j] = long_board2[h]
            h+=1
            
    return our_board


In [3]:
def letter_to_number(letter):
    letter = str(letter)
    color = 1
    if letter.isupper(): color = -1
    return  color*((1*(letter.lower() == 'p'))+(2*(letter.lower() == 'n'))\
            +(3*(letter.lower() == 'b'))+(4*(letter.lower() == 'r'))\
            +(5*(letter.lower() == 'q'))+(6*(letter.lower() == 'k')))

In [4]:
def board_to_nums(board):
    b2 = board.copy()
    for i in range(0,8):
        for j in range(0,8):
            b2[i][j] = letter_to_number(board[i][j])
    return b2

In [5]:
def board_to_12_array(board):
    ta = [[],[],[],[],[],[],[],[],[],[],[],[]]
    # if its a 0, add 0 to all 12 arrays
    # if a 1,2,3,4,5,6 or negatives, add a 1 to the corresponding array and 0 to all others
    
    b = board.copy()
    for i in range(0,8):
        for m in range(0,12):
            ta[m].append([])
        for j in range(0,8):
            # blank space
            if b[i][j] == 0:
                for k in range(0,12):
                    ta[k][i].append(0)
            # piece is white
            elif b[i][j] < 0:
                ta[-1*(b[i][j])+5][i].append(1)
                for k in range(0,12):
                    if k != (-1*(b[i][j])+5):
                        ta[k][i].append(0)
            # piece is black   
            else:
                ta[b[i][j]-1][i].append(1)
                for k in range(0,12):
                    if k != (b[i][j]-1):
                        ta[k][i].append(0)
        
            
    return ta

In [34]:
# array to hold all input positions
boards = []
num_games = 1000
gamepgn=open("lichess_12_2022.pgn",'r')
#game = chess.pgn.read_game(gamepgn)
#board = game.board()

# go through first 1000 games from dataset
for i in range(0,num_games):
    game = chess.pgn.read_game(gamepgn)
    board = game.board()
    # after reading the next game, take each position's FEN and convert to board array
    x = 1
    for move in game.mainline_moves():
        if x > 0:
            b = {}
            b['board'] = board_to_nums(board_fen_to_8x8(board.board_fen()))
            b['oneHboards'] = board_to_12_array(board_to_nums(board_fen_to_8x8(board.board_fen())))
            boards.append(b)
        board.push(move)
        x = x * (-1)
boards = pd.DataFrame(boards)

print(len(boards))
print(boards.loc[0]["board"][7])
print(boards.loc[0]["oneHboards"])

test_boards = []

# generate test dataset for later
for i in range(0,100):
    game = chess.pgn.read_game(gamepgn)
    board = game.board()
    x = 1
    for move in game.mainline_moves():
        if x > 0:
            test_boards.append(board.board_fen())
        board.push(move)
        x = x * (-1)


43418
[-4, -2, -3, -5, -6, -3, -2, -4]
[[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0,

In [13]:
rows = []
with open("lichess_12_2022.pgn") as pgn:
    for game in tqdm.tqdm(range(num_games)):
        game = chess.pgn.read_game(pgn)
        x = 1
        for move in game.mainline_moves():
            if x > 0:
                u=list(str(move.uci()).strip(""))
                squares=[]
                for i in range(0,4):
                    if u[i].isnumeric():
                        squares.append(float(u[i])/8)
                    else:
                        squares.append(float(ord(u[i])-96)/8) 
                row={}
                row['moves'] = squares
                row['start_row'] = squares[0]
                row['start_col'] = squares[1]
                row['end_row'] = squares[2]
                row['end_col'] = squares[3]
                rows.append(row)
            x = x * (-1)
                
moves = pd.DataFrame(rows)

master = pd.concat([boards, moves], axis=1, join="inner")
print(master)

master.to_csv("loadedgames.csv", index = False)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 734.76it/s]


                                                   board   
0      [[4, 2, 3, 5, 6, 3, 2, 4], [1, 1, 1, 1, 1, 1, ...  \
1      [[4, 2, 3, 5, 6, 3, 2, 4], [1, 1, 1, 0, 1, 1, ...   
2      [[4, 2, 3, 5, 6, 3, 0, 4], [1, 1, 1, 0, 1, 1, ...   
3      [[4, 2, 3, 5, 6, 3, 0, 4], [1, 1, 0, 0, 1, 1, ...   
4      [[4, 2, 0, 5, 6, 3, 0, 4], [1, 1, 0, 0, 1, 1, ...   
...                                                  ...   
43413  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, ...   
43414  [[0, 0, 0, 0, 0, 0, 0, 0], [-6, 0, 0, 0, 0, 0,...   
43415  [[0, -6, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0,...   
43416  [[0, 0, 0, 0, 0, 0, 0, 0], [0, -6, 0, 0, 0, 0,...   
43417  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...   

                                              oneHboards   
0      [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1,...  \
1      [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1,...   
2      [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1,...   
3      [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 

In [10]:
# model attempt 1
inp = keras.layers.Input(shape=(12,8,8))

m = keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding="same", activation='relu', name="conv1")(inp)
m = keras.layers.MaxPooling2D(pool_size=(2,2), padding="valid", name="mPool1")(m)
m = keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding="same", activation='relu', name="conv2")(m)
m = keras.layers.MaxPooling2D(pool_size=(2,2), padding="valid", name="mPool2")(m)
m = keras.layers.Flatten()(m)
m = keras.layers.Dense(64, activation='relu')(m)
output_move = keras.layers.Dense(4, activation='sigmoid')(m)

model = keras.Model(inputs=inp, outputs=output_move, name="pawn_star_1")
model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.mse, metrics=None)
model.summary()

Model: "pawn_star_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12, 8, 8)]        0         
                                                                 
 conv1 (Conv2D)              (None, 12, 8, 64)         4672      
                                                                 
 mPool1 (MaxPooling2D)       (None, 6, 4, 64)          0         
                                                                 
 conv2 (Conv2D)              (None, 6, 4, 64)          36928     
                                                                 
 mPool2 (MaxPooling2D)       (None, 3, 2, 64)          0         
                                                                 
 flatten_1 (Flatten)         (None, 384)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                

In [11]:
train_boards = master["oneHboards"].to_list()
train_moves = master["moves"].to_list()

#earlystop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=250, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

callback = keras.callbacks.ModelCheckpoint(filepath="checkpoints/cp-{epoch:04d}.hdf5",
                                           verbose=1, 
                                           save_freq=2000, 
                                           save_weights_only=True, 
                                           #save_best_only=True,
                                           monitor="val_loss", 
                                           mode="max")

inpx = tf.convert_to_tensor(train_boards)
inpy = tf.convert_to_tensor(train_moves)
model.fit(x=inpx, 
          y=inpy, 
          batch_size=1000, 
          epochs=10, 
          callbacks=[callback],
          validation_split=0.1)

Epoch 1/10
40/40 [==============================] - 2s 45ms/step - loss: 0.0524 - val_loss: 0.0502
Epoch 2/10
40/40 [==============================] - 2s 44ms/step - loss: 0.0483 - val_loss: 0.0471
Epoch 3/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0454 - val_loss: 0.0449
Epoch 4/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0439 - val_loss: 0.0442
Epoch 5/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0432 - val_loss: 0.0435
Epoch 6/10
40/40 [==============================] - 2s 44ms/step - loss: 0.0425 - val_loss: 0.0433
Epoch 7/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0419 - val_loss: 0.0437
Epoch 8/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0414 - val_loss: 0.0433
Epoch 9/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0409 - val_loss: 0.0433
Epoch 10/10
40/40 [==============================] - 2s 44ms/step - loss: 0.0404 - val_loss: 0.0436


In [38]:
# evaluate random position
stockfish = Stockfish(path="stockfish15.1/stockfish-windows-2022-x86-64-avx2.exe")

AI = 0
IM = 0
Random = 0

boards = random.sample(range(1, len(test_boards)), 500)

for i in boards:
    AI_board = chess.Board(test_boards[i])
    Random_board = chess.Board(test_boards[i])
    curr_eval = stockfish.get_evaluation()["value"]
    
    # try AI eval
    pos = board_to_12_array(board_to_nums(board_fen_to_8x8(AI_board.board_fen())))
    pos = np.expand_dims(pos, 0)
    move = np.array(model(pos))
    move = move * 8
    x = 1
    m = ""
    for e in range(len(move[0])):
        temp = move[0][e]
        temp = int(round(temp,0))
        if temp == 0:
            temp = 1
        if x > 0:
            temp = chr(96+int(temp))
        x = x * (-1)
        m += str(temp)
    if m[0:2] == m[2:4]:
        m = "0000"
    m = chess.Move.from_uci(m)
    move = m
    
    if(move in AI_board.legal_moves):
        AI_board.push(move)
        AI_eval = stockfish.get_evaluation()["value"]
        
        rmove = random.choice([rmove for rmove in board.legal_moves])
        Random_board.push(move)
        Random_eval = stockfish.get_evaluation()["value"]
        
        #print("Curr:   " + str(curr_eval))
        #print("AI:     " + str(AI_eval))
        #print("Random: " + str(Random_eval))
        
        if (curr_eval - AI_eval) <= (curr_eval - Random_eval):
            AI += 1
        else:
            Random += 1

    else:
        IM += 1
        
print("IM:     " + str(IM))
print("AI:     " + str(AI))
print("Random: " + str(Random))

IM:     440
AI:     38
Random: 22


In [22]:
#make move on chess object based on given move
def make_move(move):

    #make move if move is legal, else return "not legal"
    if(move in board.legal_moves):
        board.push(move)
    else:
        return("not legal")
    
    #check if game has ended and return outcome else return move made
    if(board.is_game_over()):
        return(board.outcome())
    else:
        return(move)
    
def choose_random_move(move):
    for legal_move in board.legal_moves:
        # check for something with 3 matches:
        legal_move = str(legal_move)
        if (move[0] == legal_move[0] and move[1] == legal_move[1] and move[2] == legal_move[2]) or (move[0] == legal_move[0] and move[2] == legal_move[2] and move[3] == legal_move[3]) or(move[0] == legal_move[0] and move[1] == legal_move[1] and move[3] == legal_move[3]) or(move[1] == legal_move[1] and move[2] == legal_move[2] and move[3] == legal_move[3]):
            return str(legal_move)
        else:
            return str(random.choice([move for move in board.legal_moves]))
           
           
    
stockfish = Stockfish(path="stockfish15.1/stockfish-windows-2022-x86-64-avx2.exe")
w = 0
b = 0
d = 0
AI_t = 0
Random_t = 0
num_runs = 1
for g in range(num_runs):
    #chess object
    board = chess.Board()

    #play game with random moves
    white = 1
    while not board.is_game_over():
        if white > 0:
            pos = board_to_12_array(board_to_nums(board_fen_to_8x8(board.board_fen())))
            pos = np.expand_dims(pos, 0)
            move = np.array(model(pos))
            move = move * 8
            x = 1
            m = ""
            for e in range(len(move[0])):
                temp = move[0][e]
                temp = int(round(temp,0))
                if temp == 0:
                    temp = 1
                if x > 0:
                    temp = chr(96+int(temp))
                x = x * (-1)
                m += str(temp)
            if m[0:2] == m[2:4]:
                m = "0000"
            m = chess.Move.from_uci(m)
            move = m

            gameStatus = make_move(move)
            if gameStatus == "not legal":
                Random_t += 1
                move = choose_random_move(str(move))
                move = chess.Move.from_uci(move)
                gameStatus = make_move(move)
                #print("Random White moves")  
            else:
                AI_t += 1
                #print("AI White moves:")
        else:
            move = random.choice([move for move in board.legal_moves])
            gameStatus = make_move(move)
            #print("Black moves:")

        white = white * (-1)
        #stockfish.make_moves_from_current_position([str(move)])
        #print(gameStatus)
        #print(stockfish.get_evaluation())
        #print(board)
    outcome = board.outcome()
    if outcome.winner == chess.WHITE:
        w += 1
    elif outcome.winner == chess.BLACK:
        b += 1
    else:
        d += 1
print(w)
print(b)
print(d)
print(AI_t)
print(Random_t)


0
1
0
3
143


In [ ]:
# playing chess AI ourselves:

#chess object
board = chess.Board()

#play game with random moves
white = 1
while not board.is_game_over():
    if white > 0:
        pos = board_to_12_array(board_to_nums(board_fen_to_8x8(board.board_fen())))
        pos = np.expand_dims(pos, 0)
        move = np.array(model(pos))
        print(move)
        move = move * 8
        print(move)
        x = 1
        m = ""
        for e in range(len(move[0])):
            temp = move[0][e]
            temp = int(round(temp,0))
            if temp == 0:
                temp = 1
            if x > 0:
                temp = chr(96+int(temp))
            x = x * (-1)
            m += str(temp)
        if m[0:2] == m[2:4]:
            m = "0000"
        m = chess.Move.from_uci(m)
        move = m
        gameStatus = make_move(move)
        if gameStatus == "not legal":
            Random_t += 1
            move = choose_random_move(str(move))
            move = chess.Move.from_uci(move)
            gameStatus = make_move(move)
            print("Random White moves")  
        else:
            AI_t += 1
            print("AI White moves:")
    else:
        move = input("Enter move: (e.g. e7e5)")
        move = chess.Move.from_uci(move)
        gameStatus = make_move(move)
        print("Black moves:")
        
    print(gameStatus)
    print(board)
        
    white = white * (-1) 

[[0.59309036 0.23613931 0.57772934 0.4839564 ]]
[[4.744723  1.8891145 4.6218348 3.8716512]]
AI White moves:
e2e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Enter move: (e.g. e7e5)e7e5
Black moves:
e7e5
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Enter move: (e.g. e7e5)g1f3
Black moves:
g1f3
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K B . R
Enter move: (e.g. e7e5)g8f6
Black moves:
g8f6
r n b q k b . r
p p p p . p p p
. . . . . n . .
. . . . p . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K B . R
Enter move: (e.g. e7e5)f1d3
Black moves:
f1d3
r n b q k b . r
p p p p . p p p
. . . . . n . .
. . . . p . . .
. . . . P . . .
. . . B . N . .
P P P P . P P P
R N B Q K . . R
Enter move: (e.g. e7e5)b8c6
Black moves:
b8c6
r . b q k b . r
p